In [20]:
pwd

'/home/wsuser/work'

In [21]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0

     |████████████████████████████████| 312 kB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 10.2 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow==1.14.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1,

In [22]:

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagon=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagon=ImageDataGenerator(rescale=1./255)

In [24]:


import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='TEBrmmV0ooXCLPdmM9R92zsWC9kCsCpRHiCu30HdSytc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naturaldisastermodel-donotdelete-pr-srdtyfcerua7wt'
object_key = 'dataset-20221117T143546Z-001.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [25]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [26]:
import os
filenames = os.listdir('/home/wsuser/work/dataset/train_set')

In [27]:
pwd

'/home/wsuser/work'

In [28]:
x_train=train_datagon.flow_from_directory('/home/wsuser/work/dataset/train_set',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='categorical')
x_test=test_datagon.flow_from_directory('/home/wsuser/work/dataset/test_set',target_size=(64,64),batch_size=5,color_mode='rgb',class_mode='categorical')


Found 750 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [29]:
x_train.class_indices

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}

In [30]:
model=Sequential()

In [31]:
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

In [32]:
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=4,activation='softmax'))


In [33]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

In [35]:
model.fit_generator(generator=x_train,steps_per_epoch=len(x_train),epochs=20,validation_data=x_test,validation_steps=len(x_test))

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1963: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
150/150 [==============================] - 41s 273ms/step - loss: 1.1909 - accuracy: 0.4640 - val_loss: 0.9606 - val_accuracy: 0.6061
Epoch 2/20
150/150 [==============================] - 39s 263ms/step - loss: 0.8433 - accuracy: 0.6680 - val_loss: 0.8321 - val_accuracy: 0.6566
Epoch 3/20
150/150 [==============================] - 40s 266ms/step - loss: 0.6793 - accuracy: 0.7240 - val_loss: 0.7982 - val_accuracy: 0.7222
Epoch 4/20
150/150 [==============================] - 43s 286ms/step - loss: 0.5713 - accuracy: 0.7813 - val_loss: 1.1609 - val_accuracy: 0.6111
Epoch 5/20
150/150 [==============================] - 41s 272ms/step - loss: 0.5385 - accuracy: 0.7853 - val_loss: 0.6987 - val_accuracy: 0.7424
Epoch 6/20
150/150 [==============================] - 41s 271ms/step - loss: 0.5075 - accuracy: 0.8120 - val_loss: 0.6089 - val_accuracy: 0.7929
Epoch 7/20
150/150 [==============================] - 40s 265ms/step - loss: 0.4541 - accuracy: 0.8360 - val_loss: 0.9015 - val_ac

In [36]:
model.save('disaster.h5')

In [37]:
!tar -zcvf image-classification-model_new.tgz disaster.h5

disaster.h5


In [38]:
ls -1

dataset/
disaster.h5
image-classification-model_new.tgz


In [39]:
!pip install watson-machine-learning-client --upgrade

In [40]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "dSP4DoIWTICCJT-ZrUO1jZL2lPDv2DT_5pdAttZSxKrT"
}
client = APIClient(wml_credentials)

In [41]:
client = APIClient(wml_credentials)

In [42]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [43]:
space_uid = guid_from_space_name(client,'Natural_disaster_model')
print("Space UID = "+space_uid)

Space UID = 986bcb37-8cb0-4b1e-ad01-9a41772f53b1


In [44]:
client.set.default_space(space_uid)

'SUCCESS'

In [45]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [46]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")


software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [47]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={client.repository.ModelMetaNames.NAME:"CNN",
                                                                                                      client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid,
                                                                                                    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7"

})
        
model_id = client.repository.get_model_id(model_details)

In [48]:
model_id

'2d8f4b5d-699f-4098-834d-05f2a6713467'

In [49]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [50]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-18T13:27:09.511Z',
  'id': '2d8f4b5d-699f-4098-834d-05f2a6713467',
  'modified_at': '2022-11-18T13:27:12.384Z',
  'name': 'CNN',
  'owner': 'IBMid-667000CV8L',
  'resource_key': '8b2d7cce-fd86-4ec1-9cc7-d0b58d012b7b',
  'space_id': '986bcb37-8cb0-4b1e-ad01-9a41772f53b1'},
 'system': {'warnings': []}}

In [ ]:
index=['Cyclone','Earthquake','Flood','Wildfire']
y=np.argmax(model.predict(x),axis=1)
print(index[int(y)])

1/1 [==============================] - 0s 17ms/step
Earthquake
